In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from keras import layers
from keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [11]:
dataset1=pd.read_csv('/content/train_data_covid.csv')
dataset2=pd.read_csv('/content/test_data_covid.csv')
new_column_data = list(range(1, 3111))
dataset2.to_csv('testset.csv', index=False)
dataset2['Deaths'] = new_column_data
encoder=LabelEncoder()
combined_dataset = pd.concat([dataset1, dataset2], ignore_index=True)
combined_dataset['State/UnionTerritory'] = encoder.fit_transform(combined_dataset['State/UnionTerritory'])
dataset1_encoded = combined_dataset.iloc[:15001]
dataset2_encoded = combined_dataset.iloc[15001:]
dataset1_encoded.to_csv('/content/train_data_covid.csv', index=False)
dataset2_encoded.to_csv('/content/test_data_covid.csv', index=False)


In [12]:
train=pd.read_csv('/content/train_data_covid.csv')
import datetime as dt
train['Date'] = pd.to_datetime(train['Date'])
train['Date']=train['Date'].map(dt.datetime.toordinal)
train['ConfirmedForeignNational']=train['ConfirmedForeignNational'].replace('-',0)
train['ConfirmedIndianNational']=train['ConfirmedIndianNational'].replace('-',0)
# Transform the same categorical columns in the second dataset
train.head(15000)


,Sno,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed,OxygenTanks,PopulationDensityPerSqKm
0,1,737454,6:00 PM,22,1,0,0,0,1,8262,867
1,2,737455,6:00 PM,22,1,0,0,0,1,9136,61
2,3,737456,6:00 PM,22,2,0,0,0,2,3826,316
3,4,737457,6:00 PM,22,3,0,0,0,3,9789,806
4,5,737458,6:00 PM,22,3,0,0,0,3,882,369
...,...,...,...,...,...,...,...,...,...,...,...
14995,14996,737927,8:00 AM,23,0,0,14741,165,16452,5849,773
14996,14997,737927,8:00 AM,24,0,0,3633,14,4767,4046,490
14997,14998,737927,8:00 AM,25,0,0,629741,6992,731385,1195,595
14998,14999,737927,8:00 AM,27,0,0,4826371,81486,5378452,9289,195


In [13]:
test=pd.read_csv('/content/test_data_covid.csv')
import datetime as dt
test['Date'] = pd.to_datetime(test['Date'])
test['Date']=test['Date'].map(dt.datetime.toordinal)
test['ConfirmedIndianNational']=test['ConfirmedIndianNational'].replace('-',0)
test['ConfirmedForeignNational']=test['ConfirmedForeignNational'].replace('-',0)
test.head()


,Sno,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed,OxygenTanks,PopulationDensityPerSqKm
0,15001,737927,8:00 AM,30,0,0,18478,1,23332,191,514
1,15002,737927,8:00 AM,31,0,0,6687,2,8829,1111,563
2,15003,737927,8:00 AM,32,0,0,13767,3,18076,9896,577
3,15004,737927,8:00 AM,33,0,0,514532,4,612224,6091,615
4,15005,737927,8:00 AM,34,0,0,65689,5,84506,220,573


In [15]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
import random
from sklearn.ensemble import GradientBoostingRegressor

X_train=train[['Confirmed','PopulationDensityPerSqKm','Date','Cured','State/UnionTerritory']]
y_train=train['Deaths']

x_test=test[['Confirmed','PopulationDensityPerSqKm','Date','Cured','State/UnionTerritory']]
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Apply the same scaling to the testing data
X_test_scaled = scaler.transform(x_test)

poly_features = PolynomialFeatures(degree=4)
X_poly = poly_features.fit_transform(X_train_scaled)
x_test_poly = poly_features.transform(X_test_scaled)
model=GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=38, random_state=22)
model.fit(X_poly,y_train)
# Make predictions on the test data
y_pred = model.predict(x_test_poly)
act = y_train.head(3110)
# Calculate mean squared error
mse = mean_squared_error(act, y_pred)
print("Mean Squared Error:",mse)

Mean Squared Error: 540318566.3231946


In [16]:
test['Deaths']=y_pred
test['Deaths'] = test['Deaths'].apply(lambda x: max(0, x))
test['Deaths'] = test['Deaths'].apply(lambda x: round(x))
# Write the updated DataFrame back to a CSV fi
test


,Sno,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed,OxygenTanks,PopulationDensityPerSqKm
0,15001,737927,8:00 AM,30,0,0,18478,320,23332,191,514
1,15002,737927,8:00 AM,31,0,0,6687,24,8829,1111,563
2,15003,737927,8:00 AM,32,0,0,13767,194,18076,9896,577
3,15004,737927,8:00 AM,33,0,0,514532,8521,612224,6091,615
4,15005,737927,8:00 AM,34,0,0,65689,1117,84506,220,573
...,...,...,...,...,...,...,...,...,...,...,...
3105,18106,738013,8:00 AM,39,0,0,638410,6548,650353,8796,659
3106,18107,738013,8:00 AM,41,0,0,77811,1070,80660,9330,488
3107,18108,738013,8:00 AM,44,0,0,334650,6787,342462,6290,686
3108,18109,738013,8:00 AM,43,0,0,1685492,61632,1708812,723,115


In [17]:
predictions = pd.DataFrame(test, columns=['Sno','Deaths'])
predictions.to_csv('sample_submission.csv', index = False)